In [1]:
# # in google colab uncomment this

# import os

# os.system('apt-get install -y xvfb')
# os.system('wget https://raw.githubusercontent.com/yandexdataschool/Practical_DL/fall18/xvfb -O ../xvfb')
# os.system('apt-get install -y python-opengl ffmpeg')
# os.system('pip install pyglet==1.2.4')

# os.system('python -m pip install -U pygame --user')

# print('setup complete')

# XVFB will be launched if you run on a server
import os
if type(os.environ.get("DISPLAY")) is not str or len(os.environ.get("DISPLAY")) == 0:
    !bash ../xvfb start
    %env DISPLAY = : 1

# Implementing Advantage-Actor Critic (A2C)

In this notebook you will implement Advantage Actor Critic algorithm that trains on a batch of Atari 2600 environments running in parallel. 

Firstly, we will use environment wrappers implemented in file `atari_wrappers.py`. These wrappers preprocess observations (resize, grayscal, take max between frames, skip frames and stack them together) and rewards. Some of the wrappers help to reset the environment and pass `done` flag equal to `True` when agent dies.
File `env_batch.py` includes implementation of `ParallelEnvBatch` class that allows to run multiple environments in parallel. To create an environment we can use `nature_dqn_env` function. Note that if you are using 
PyTorch and not using `tensorboardX` you will need to implement a wrapper that will log **raw** total rewards that the *unwrapped* environment returns and redefine the implemention of `nature_dqn_env` function here. 



In [2]:
import numpy as np
from atari_wrappers import nature_dqn_env


env = nature_dqn_env("SpaceInvadersNoFrameskip-v4", nenvs=8)
obs = env.reset()
assert obs.shape == (8, 84, 84, 4) # 8 个环境并行，84x84单色，4帧合并 
assert obs.dtype == np.uint8
print(env.action_space) # Discrete(6) 0-5
print(env.action_space.sample()) # 8 个环境并行 ,每个连续按动同一个动作 

entry_point is :gym.envs.atari:AtariEnv
entry_point is :gym.envs.atari:AtariEnv


/home/jim/.local/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(require = False)


entry_point is :gym.envs.atari:AtariEnv


/home/jim/.local/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(require = False)
/home/jim/.local/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(require = False)


entry_point is :gym.envs.atari:AtariEnv
entry_point is :gym.envs.atari:AtariEnv
entry_point is :gym.envs.atari:AtariEnv


/home/jim/.local/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(require = False)
/home/jim/.local/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(require = False)


entry_point is :gym.envs.atari:AtariEnv
entry_point is :gym.envs.atari:AtariEnv


/home/jim/.local/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(require = False)
/home/jim/.local/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(require = False)
/home/jim/.local/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(require = False)


[4 5 0 3 3 3 1 3]


Next, we will need to implement a model that predicts logits and values. It is suggested that you use the same model as in [Nature DQN paper](https://web.stanford.edu/class/psych209/Readings/MnihEtAlHassibis15NatureControlDeepRL.pdf) with a modification that instead of having a single output layer, it will have two output layers taking as input the output of the last hidden layer. **Note** that this model is different from the model you used in homework where you implemented DQN. You can use your favorite deep learning framework here. We suggest that you use orthogonal initialization with parameter $\sqrt{2}$ for kernels and initialize biases with zeros. 

In [34]:
# import tensorflow as torch
# import torch as tf
import mxnet as mx
#<Define your model here>
class my_model_class(mx.gluon.nn.HybridBlock):
    def __init__(self, **kwargs):
        super(my_model_class, self).__init__(**kwargs)
        self.net = mx.gluon.nn.HybridSequential()
        self.net.add(mx.gluon.nn.Conv2D(channels=4,layout="NCHW",kernel_size=3))
        self.net.add(mx.gluon.nn.Conv2D(channels=16,layout="NCHW",kernel_size=3))
        self.net.add(mx.gluon.nn.Conv2D(channels=16,layout="NCHW",kernel_size=2))
        self.net.add(mx.gluon.nn.Conv2D(channels=8,layout="NCHW",kernel_size=2))
        self.net.add(mx.gluon.nn.Conv2D(channels=4,layout="NCHW",kernel_size=3))
        self.net.add(mx.gluon.nn.AvgPool2D(layout="NCHW",pool_size=2))
        #
        self.actions_conv2d = mx.gluon.nn.Conv2D(channels=1,layout="NCHW",kernel_size=3)
        self.actions_desen = mx.gluon.nn.Dense(6,activation='relu')
        #
        self.values_conv2d = mx.gluon.nn.Conv2D(channels=1,layout="NCHW",kernel_size=3)
        self.values_desen = mx.gluon.nn.Dense(1,activation='sigmoid')
    def hybrid_forward(self,F,x):
        middle_x = self.net(x)
        y_actions = self.actions_conv2d(middle_x)
        y_actions = self.actions_desen(y_actions)
        y_values = self.values_conv2d(middle_x)
        y_values = self.values_desen(y_values)
        return y_actions, y_values

model = my_model_class()
print(model)
model.initialize(init=mx.init.Xavier())
model.hybridize()

my_model_class(
  (net): HybridSequential(
    (0): Conv2D(None -> 4, kernel_size=(3, 3), stride=(1, 1))
    (1): Conv2D(None -> 16, kernel_size=(3, 3), stride=(1, 1))
    (2): Conv2D(None -> 16, kernel_size=(2, 2), stride=(1, 1))
    (3): Conv2D(None -> 8, kernel_size=(2, 2), stride=(1, 1))
    (4): Conv2D(None -> 4, kernel_size=(3, 3), stride=(1, 1))
    (5): AvgPool2D(size=(2, 2), stride=(2, 2), padding=(0, 0), ceil_mode=False, global_pool=False, pool_type=avg, layout=NCHW)
  )
  (actions_conv2d): Conv2D(None -> 1, kernel_size=(3, 3), stride=(1, 1))
  (actions_desen): Dense(None -> 6, Activation(relu))
  (values_conv2d): Conv2D(None -> 1, kernel_size=(3, 3), stride=(1, 1))
  (values_desen): Dense(None -> 1, Activation(sigmoid))
)


You will also need to define and use a policy that wraps the model. While the model computes logits for all actions, the policy will sample actions and also compute their log probabilities.  `policy.act` should return a dictionary of all the arrays that are needed to interact with an environment and train the model.
 Note that actions must be an `np.ndarray` while the other
tensors need to have the type determined by your deep learning framework. 

In [40]:
class Policy:
  def __init__(self, model):
    self.model = model
    
  def act(self, inputs):
    #<Implement policy by calling model, sampling actions and computing their log probs>
    # Should return a dict containing keys ['actions', 'logits', 'log_probs', 'values'].
    y_actions, y_values = self.model(inputs)
    #print(y_actions.shape)
    #print(y_values.shape)
    rel = {}
    logits = mx.nd.softmax(y_actions,axis=-1)
    #print(logits.shape)
    rel['logits'] = logits
    m,n = logits.shape
    rel['actions'] = []
    for i in np.arange(m):
        probs = logits[i,:].asnumpy()
        #print(probs)
        rel['actions'].append(np.random.choice(np.array([0,1,2,3,4,5]),1,p = probs)[0])
    #print(rel['actions'])
    log_probs = mx.nd.log_softmax(y_actions,axis=-1)
    rel['log_probs'] = log_probs
    rel['values'] = y_values
    return rel

test_policy = Policy(model)
inputs = mx.nd.random.uniform(low=0, high=255,shape = (8,4,48,48),dtype='float32')
print(inputs.shape)
test_policy.act(inputs)

(8, 4, 48, 48)


{'logits': 
 [[7.8249885e-31 9.1588831e-01 7.8249885e-31 2.9682927e-27 8.4111705e-02
   7.8249885e-31]
  [1.0681485e-19 3.4122545e-02 3.9990695e-23 1.1608352e-13 9.6587747e-01
   3.9990695e-23]
  [1.6666667e-01 1.6666667e-01 1.6666667e-01 1.6666667e-01 1.6666667e-01
   1.6666667e-01]
  [9.6887113e-31 1.0000000e+00 5.0957469e-31 5.0957469e-31 1.0348655e-30
   5.0957469e-31]
  [3.4279261e-02 2.1384282e-07 2.1384282e-07 2.1384282e-07 9.6571976e-01
   2.1384282e-07]
  [2.3978490e-09 2.3978490e-09 2.3978490e-09 1.0000000e+00 2.3978490e-09
   2.3978490e-09]
  [1.3233224e-01 3.9723707e-14 3.9723707e-14 8.6766779e-01 3.9723707e-14
   3.9723707e-14]
  [1.7453045e-02 1.7453045e-02 1.7453045e-02 4.7650632e-01 4.5368150e-01
   1.7453045e-02]]
 <NDArray 8x6 @cpu(0)>, 'actions': [4, 4, 5, 1, 4, 3, 3, 4], 'log_probs': 
 [[-6.9322815e+01 -8.7860882e-02 -6.9322815e+01 -6.1081810e+01
   -2.4756095e+00 -6.9322815e+01]
  [-4.3683189e+01 -3.3777969e+00 -5.1573395e+01 -2.9784466e+01
   -3.4718312e-02 -5.157

Next will pass the environment and policy to a runner that collects partial trajectories from the environment. 
The class that does is is already implemented for you.

In [ ]:
from runners import EnvRunner

This runner interacts with the environment for a given number of steps and returns a dictionary containing
keys 

* 'observations' 
* 'rewards' 
* 'resets'
* 'actions'
* all other keys that you defined in `Policy`

under each of these keys there is a python `list` of interactions with the environment of specified length $T$ &mdash; the size of partial trajectory. 

To train the part of the model that predicts state values you will need to compute the value targets. 
Any callable could be passed to `EnvRunner` to be applied to each partial trajectory after it is collected. 
Thus, we can implement and use `ComputeValueTargets` callable. 
The formula for the value targets is simple:

$$
\hat v(s_t) = \sum_{t'=0}^{T - 1}\gamma^{t'}r_{t+t'} + \gamma^T \hat{v}(s_{t+T}),
$$

In implementation, however, do not forget to use 
`trajectory['resets']` flags to check if you need to add the value targets at the next step when 
computing value targets for the current step. You can access `trajectory['state']['latest_observation']`
to get last observations in partial trajectory &mdash; $s_{t+T}$.

In [ ]:
class ComputeValueTargets:
  def __init__(self, policy, gamma=0.99):
    self.policy = policy
    
  def __call__(self, trajectory):
    # This method should modify trajectory inplace by adding 
    # an item with key 'value_targets' to it. 
    <Compute value targets for a given partial trajectory>

After computing value targets we will transform lists of interactions into tensors
with the first dimension `batch_size` which is equal to `T * nenvs`, i.e. you essentially need
to flatten the first two dimensions. 

In [ ]:
class MergeTimeBatch:
  """ Merges first two axes typically representing time and env batch. """
  def __call__(self, trajectory):
    # Modify trajectory inplace. 
    <TODO: implement>

In [ ]:
model = <Create your model here>
policy = Policy(model)
runner = EnvRunner(env, policy, nsteps=5,
                   transforms=[ComputeValueTargets(),
                               MergeTimeBatch()])

Now is the time to implement the advantage actor critic algorithm itself. You can look into your lecture,
[Mnih et al. 2016](https://arxiv.org/abs/1602.01783) paper, and [lecture](https://www.youtube.com/watch?v=Tol_jw5hWnI&list=PLkFD6_40KJIxJMR-j5A1mkxK26gh_qg37&index=20) by Sergey Levine.

In [ ]:
class A2C:
  def __init__(self,
               policy,
               optimizer,
               value_loss_coef=0.25,
               entropy_coef=0.01,
               max_grad_norm=0.5):
    self.policy = policy
    self.optimizer = optimizer
    self.value_loss_coef = value_loss_coef
    self.entropy_coef = entropy_coef
    self.max_grad_norm = max_grad_norm
    
  def policy_loss(self, trajectory):
    # You will need to compute advantages here. 
    <TODO: implement>
    
  def value_loss(self, trajectory):
    <TODO: implement>
    
  def loss(self, trajectory):
    <TODO: implement>
      
  def step(self, trajectory):
    <TODO: implement>

Now you can train your model. With reasonable hyperparameters training on a single GTX1080 for 10 million steps across all batched environments (which translates to about 5 hours of wall clock time)
it should be possible to achieve *average raw reward over last 100 episodes* (the average is taken over 100 last 
episodes in each environment in the batch) of about 600. You should plot this quantity with respect to 
`runner.step_var` &mdash; the number of interactions with all environments. It is highly 
encouraged to also provide plots of the following quantities (these are useful for debugging as well):

* [Coefficient of Determination](https://en.wikipedia.org/wiki/Coefficient_of_determination) between 
value targets and value predictions
* Entropy of the policy $\pi$
* Value loss
* Policy loss
* Value targets
* Value predictions
* Gradient norm
* Advantages
* A2C loss

For optimization we suggest you use RMSProp with learning rate starting from 7e-4 and linearly decayed to 0, smoothing constant (alpha in PyTorch and decay in TensorFlow) equal to 0.99 and epsilon equal to 1e-5.

In [ ]:
a2c = <Create instance of the algorithm> 

<Write your training loop>